In [ ]:
#instalar o Streamlit e o cloudflared (que vai "expor" o servidor do Colab para a internet)
!pip install -q streamlit
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

In [ ]:
#Escreve o app.py no collab para poder ter uma interface
%%writefile app.py

#importa o streamLit para podermos seguir com a programação normal no collab
import streamlit as st

# Configuração da página (Isso define o comportamento no browser)
st.set_page_config(page_title="Minha IA", layout="wide")

st.title("Construindo Interfaces com IA")

#1 - Comece pensando nas abas paa organizar as informações em seus contextos
tab_home, tab_metricas = st.tabs(["Início", "Métricas"])

#com a primeira tab comece a pensar sobre o input e output
with tab_home:
    #2 - Dentro da primeira tab, pense em colunas. É interessante separar em colunas o que preciso pedir? ou melhor manter em uma lista inteira?
    col_input, col_preview = st.columns([1, 1]) # Proporção das colunas

    with col_input:
      #3 - a partir da coluna, pense nas linhas, nas informações que você precisa pedir ou apresentar
        st.subheader("Entrada")
        upload = st.file_uploader("Suba uma imagem para análise", type=["jpg", "png"])
        prompt = st.text_area("O que a IA deve procurar?")
        botao = st.button("Analisar Agora")

    with col_preview:
        st.subheader("Saída da IA")
        if botao:
            st.success("Processamento concluído!")
            # Simulação de saída
            st.image("https://via.placeholder.com/400", caption="Resultado da Detecção")

with tab_metricas:
    #Métricas de Contexto
    st.subheader("Métricas do Modelo")
    m1, m2, m3 = st.columns(3)
    m1.metric("Precisão", "92%", "+1.5%")
    m2.metric("Tempo de Resposta", "0.8s", "-0.2s")
    m3.metric("Uso de Memória", "450MB")

In [ ]:
import subprocess
import threading
import time

def run_streamlit():
    # Roda o streamlit na porta 8501
    subprocess.Popen(["streamlit", "run", "app.py", "--server.port", "8501"])

def run_tunnel():
    # Cria o túnel da Cloudflare
    p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://localhost:8501"],
                         stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    for line in p.stdout:
        if ".trycloudflare.com" in line:
            print("\n--- SEU APP ESTÁ RODANDO NO LINK ABAIXO ---")
            print(line.split("https://")[1].strip().split(" ")[0])
            print("-------------------------------------------\n")

# Inicia o Streamlit em uma thread e o túnel em outra
threading.Thread(target=run_streamlit).start()
time.sleep(5)
run_tunnel()